In [1]:
!pip install numerapi

In [2]:
import numpy as np
import pandas as pd

import os

from IPython.display import display

import numerapi

In [3]:
# numerapiを使えばデータセットのダウンロードが簡単にできる
#インスタンス化（numerapiを使うための準備）
napi = numerapi.NumerAPI(verbosity="info")

# 現在のラウンドのデータセットをダウンロードして解凍する。
napi.download_current_dataset(unzip=True)

2021-10-09 18:37:57,373 INFO numerapi.utils: starting download
./numerai_dataset_285.zip: 425MB [00:26, 16.3MB/s]                           
2021-10-09 18:38:23,429 INFO numerapi.base_api: unzipping file...


'./numerai_dataset_285.zip'

In [4]:
# numerai_dataset_321/numerai_training_data.csv でトレーニングデータのファイル名
# numerai_dataset_321/numerai_tournament_data.csv でトーナメントデータのファイル名

# まずは現在のトーナメントのラウンド数を取得(int型)
current_ds = napi.get_current_round()
print(current_ds)

# ここはnumerai_dataset_321のようなパスを得るため
latest_round = os.path.join('numerai_dataset_'+str(current_ds))
print(latest_round)

285
numerai_dataset_285


In [5]:
print("# データの読み込み中...")
# トレーニングデータをCSVから読み込む。　set_indexでどの列をindexにするか？を決める
# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）
train = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv"))  #.set_index("id")
train_x = train.drop(['id', 'era', 'data_type', 'target'], axis=1)
train_y = train['target']

# データの読み込み中...


In [6]:
# トーナメントデータをCSVから読み込む。
test = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv"))  #.set_index("id")
test_x = test.drop(['id', 'era', 'data_type', 'target'], axis=1)

feature_names = [f for f in train_x.columns if "feature" in f]

In [7]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
train_x.iloc[tr_idx], train_x.iloc[va_idx]
train_y.iloc[tr_idx], train_y.iloc[va_idx]

(0         0.50
 1         0.25
 2         0.25
 3         0.25
 4         0.75
           ... 
 501801    0.50
 501802    0.50
 501803    0.50
 501804    0.75
 501805    0.25
 Name: target, Length: 376356, dtype: float64, 6         0.25
 7         0.25
 9         0.75
 16        0.50
 17        0.50
           ... 
 501789    0.25
 501790    1.00
 501794    0.25
 501806    0.50
 501807    0.50
 Name: target, Length: 125452, dtype: float64)

In [8]:
train_x

,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,feature_charisma27,feature_charisma28,...,feature_wisdom7,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46
0,0.00,0.50,0.25,0.00,0.50,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.50,1.00,0.50,0.00,0.50,0.50,0.00,0.00,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.50,0.50,0.75,0.50,0.25,0.75,0.75,...,0.75,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.50,0.25,0.25,0.75,0.50,1.00,0.50,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.50,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75
1,0.00,0.00,0.00,0.25,0.50,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.50,0.50,0.50,0.50,0.25,0.25,0.50,0.00,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.50,0.25,0.75,0.50,0.50,0.50,0.75,...,1.00,0.25,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.00,0.25,1.00,0.50,1.00,1.00,0.50,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.50,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00
2,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.50,0.50,1.00,0.50,0.50,0.50,0.25,0.00,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.50,0.50,0.75,0.50,0.50,0.25,0.75,...,0.75,0.25,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.50,1.00,0.00,1.00,1.00,0.50,1.00,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.00,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75
3,1.00,0.00,0.00,0.50,0.50,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.00,0.50,0.50,0.00,1.00,0.00,0.75,0.00,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,...,1.00,1.00,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.00,0.75,1.00,0.75,1.00,0.75,1.00,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.50,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00
4,0.25,0.25,0.25,0.25,0.00,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.00,0.50,0.50,0.25,0.00,0.50,0.00,0.50,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.50,0.25,...,0.75,1.00,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.50,1.00,0.50,0.75,0.25,0.50,0.00,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.50,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501803,0.50,0.50,0.25,0.00,0.00,0.50,0.75,0.00,0.75,0.75,0.25,0.00,1.00,0.00,0.75,0.00,0.50,1.00,0.25,0.25,0.00,0.50,0.75,0.25,0.00,0.00,0.25,1.00,0.00,0.75,1.00,0.00,0.0

In [9]:
train_y

0         0.50
1         0.25
2         0.25
3         0.25
4         0.75
          ... 
501803    0.50
501804    0.75
501805    0.25
501806    0.50
501807    0.50
Name: target, Length: 501808, dtype: float64

In [10]:
# tensorflowの警告抑制
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [11]:
# -----------------------------------
# ニューラルネットの実装
# -----------------------------------
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

In [12]:
# データのスケーリング
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
# train_y = scaler.transform(train_y)
test_x = scaler.transform(test_x)

In [13]:
# ニューラルネットモデルの構築
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [14]:
# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size = 128
epochs = 10
history = model.fit(train_x, train_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(train_x, train_y))

Epoch 1/10
3921/3921 [==============================] - 18s 4ms/step - loss: 0.6939 - accuracy: 0.0514 - val_loss: 0.6930 - val_accuracy: 0.0511
Epoch 2/10
3921/3921 [==============================] - 15s 4ms/step - loss: 0.6931 - accuracy: 0.0518 - val_loss: 0.6930 - val_accuracy: 0.0526
Epoch 3/10
3921/3921 [==============================] - 15s 4ms/step - loss: 0.6931 - accuracy: 0.0514 - val_loss: 0.6932 - val_accuracy: 0.0522
Epoch 4/10
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6931 - accuracy: 0.0513 - val_loss: 0.6930 - val_accuracy: 0.0526
Epoch 5/10
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6930 - accuracy: 0.0518 - val_loss: 0.6929 - val_accuracy: 0.0520
Epoch 6/10
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6930 - accuracy: 0.0517 - val_loss: 0.6929 - val_accuracy: 0.0515
Epoch 7/10
3921/3921 [==============================] - 15s 4ms/step - loss: 0.6929 - accuracy: 0.0520 - val_loss: 0.6927 - val_ac

In [15]:
# バリデーションデータでのスコアの確認
va_pred = model.predict(train_x)
#score = log_loss(train_y, va_pred, eps=1e-7)
#print(f'logloss: {score:.4f}')

In [16]:
# 予測
pred = model.predict(test_x)

In [17]:
# -----------------------------------
# アーリーストッピング
# -----------------------------------
from keras.callbacks import EarlyStopping

In [18]:
# アーリーストッピングの観察するroundを20とする
# restore_best_weightsを設定することで、最適なエポックでのモデルを使用する
epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [19]:
history = model.fit(train_x, train_y,
                    batch_size=batch_size, epochs=epochs,
                    verbose=1, validation_data=(train_x, train_y), callbacks=[early_stopping])
pred = model.predict(test_x)

Epoch 1/50
3921/3921 [==============================] - 15s 4ms/step - loss: 0.6927 - accuracy: 0.0530 - val_loss: 0.6926 - val_accuracy: 0.0544
Epoch 2/50
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6926 - accuracy: 0.0530 - val_loss: 0.6923 - val_accuracy: 0.0540
Epoch 3/50
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6926 - accuracy: 0.0532 - val_loss: 0.6923 - val_accuracy: 0.0543
Epoch 4/50
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6925 - accuracy: 0.0532 - val_loss: 0.6920 - val_accuracy: 0.0545
Epoch 5/50
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6924 - accuracy: 0.0536 - val_loss: 0.6918 - val_accuracy: 0.0550
Epoch 6/50
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6923 - accuracy: 0.0533 - val_loss: 0.6919 - val_accuracy: 0.0549
Epoch 7/50
3921/3921 [==============================] - 14s 4ms/step - loss: 0.6922 - accuracy: 0.0536 - val_loss: 0.6918 - val_ac

In [20]:
pred

array([[0.5005752 ],
       [0.43738025],
       [0.5897643 ],
       ...,
       [0.5005752 ],
       [0.5005752 ],
       [0.5005752 ]], dtype=float32)

In [21]:
test["prediction"] = pred/5

In [22]:
test

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,prediction
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,...,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25,0.100115
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,...,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50,0.087476
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,...,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00,0.117953
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50,0.100115
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,...,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75,0.100115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832994,nffe489171b01328,eraX,live,0.50,1.00,0.75,0.25,0.00,1.00,1.00,0.00,0.75,0.75,0.25,0.00,0.75,0.50

In [23]:
test1 = test.set_index("id")

In [24]:
test1

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,prediction
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25,0.100115
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50,0.087476
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00,0.117953
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50,0.100115
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75,0.100115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [25]:
# トーナメント名
TOURNAMENT_NAME = "nomi"

In [26]:
# tournament_data['prediction'].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")
test1['prediction'].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [27]:
# APIキーの設定
public_id = "CKTO72R47MM5TO5W4LF7OIGS2O2UVJIS"
secret_key = "54INV3JBRJNEJ6PUTIPVTO7GJ6BIVJGSEQEEJHTJGAW44NTMAZ6GSCTNYOSNYWZX"
model_id = "90684cc9-b57a-4403-9f2d-3938ffd505b5"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [28]:
# 予測の提出
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)

2021-10-09 18:56:46,148 INFO numerapi.base_api: uploading predictions...
